In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import gc
import time
import os
import sys
#sys.path.append(os.path.join(os.path.dirname(__file__), '../'))
sys.path.append('../models')

from data_frame import DataFrame
from tf_base_model import TFBaseModel # for building our customized

config = tf.ConfigProto(allow_soft_placement = True)
config.gpu_options.allow_growth=True

class DataReader(object):
    '''for reading data'''
    
    def __init__(self, data_dir):
        data_cols = [
            'item_id',
            'word_id',
            'history_length',
            'label'
        ]
        #-----------------
        # loading data
        #-----------------
        if TRACE_CODE == True:
            data_train = [np.load(os.path.join(data_dir, 'train/{}_0.npy'.format(i)), mmap_mode='r') for i in data_cols]
            data_val = [np.load(os.path.join(data_dir, 'val/{}_0.npy'.format(i)), mmap_mode='r') for i in data_cols]
            data_test = [np.load(os.path.join(data_dir, 'test/{}_0.npy'.format(i)), mmap_mode='r') for i in data_cols]
        else:
            data_train = [np.load(os.path.join(data_dir, 'train/{}.npy'.format(i)), mmap_mode='r') for i in data_cols]
            data_val = [np.load(os.path.join(data_dir, 'val/{}.npy'.format(i)), mmap_mode='r') for i in data_cols]
            data_test = [np.load(os.path.join(data_dir, 'test/{}.npy'.format(i)), mmap_mode='r') for i in data_cols]

        #------------------
        # For Testing-phase
        #------------------
        self.test_df = DataFrame(columns=data_cols, data=data_test)
        print ('loaded data')
        #------------------
        # For Training-phase
        #------------------
        self.train_df = DataFrame(columns=data_cols, data=data_train)
        self.val_df = DataFrame(columns=data_cols, data=data_val)

        #self.train_df, self.val_df = self.test_df.train_test_split(train_size=0.9)
        #self.train_df, self.val_df = self.test_df.train_test_split(train_size=0.9, random_state = 3)

        
        print ('number of training example: {}'.format(len(self.train_df)))
        print ('number of validating example: {}'.format(len(self.val_df)))
        print ('number of testing example: {}'.format(len(self.test_df)))
        
    def train_batch_generator(self, batch_size, num_epochs=100000, shuffle = True, is_test = False):
        return self.batch_generator(
            batch_size=batch_size,
            df=self.train_df,
            shuffle=shuffle,
            num_epochs=num_epochs,
            is_test=is_test
        )

    def val_batch_generator(self, batch_size, num_epochs=100000, shuffle = True, is_test = False):
        return self.batch_generator(
            batch_size=batch_size,
            df=self.val_df,
            shuffle=shuffle,
            num_epochs=num_epochs,
            is_test=is_test
        )

    def test_batch_generator(self, batch_size):
        '''All row in our dataframe need to predicted as input of second-level model'''
        return self.batch_generator(
            batch_size=batch_size,
            df=self.test_df,
            shuffle=False,
            num_epochs=1,
            is_test=True
        )
    
    def batch_generator(self, batch_size, df, shuffle=True, num_epochs=10000, is_test=False):
        '''
        df: customized DataFrame object,
        '''
        # call our customized DataFrame object method batch_generator
        batch_gen = df.batch_generator(batch_size, shuffle = shuffle, num_epochs=num_epochs, allow_smaller_final_batch=is_test)
        # batch_gen is a generator
        for batch in batch_gen:
            # what batch_gen yield is also a customized Dataframe object.
            if not is_test:
                pass
            yield batch


/home/linuxbrew/.linuxbrew/opt/python/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# CNN-based char-leve extraction reference: 1. https://github.com/IsaacChanghau/neural_sequence_labeling/blob/master/models/nns.py#L64
# https://github.com/IsaacChanghau/neural_sequence_labeling/blob/master/models/blstm_cnn_crf_model.py

In [2]:
word_id = np.load(os.path.join('../models/data/train/', '{}.npy'.format('word_id')))
history_length = np.load(os.path.join('../models/data/train/', '{}.npy'.format('history_length')))
label = np.load(os.path.join('../models/data/train/', '{}.npy'.format('label')))
item_id = np.load(os.path.join('../models/data/train/','{}.npy'.format('item_id')))
char_id = np.load(os.path.join('../models/data/train/','{}.npy'.format('char_id')))
word_lengths = np.load(os.path.join('../models/data/train/','{}.npy'.format('word_length')))
sentence = word_id[:2]
y_true = label[:2]
length = history_length[:2]
char_id_____ = char_id[:2]
word_lengths______ = word_lengths[:2]

sentence.shape, y_true.shape, length.shape, char_id_____.shape, word_lengths______.shape

((2, 36), (2, 36), (2,), (2, 36, 54), (2, 36))

In [15]:
length

array([ 8, 11], dtype=int32)

In [3]:
def get_optimizer(learning_rate, optimizer='adam'):
    '''
    It's for choosing optimizer given learning rate.
    '''
    if optimizer == 'adam':
        return tf.train.AdamOptimizer(learning_rate)
    elif optimizer == 'adagrad':
        return tf.train.AdagradOptimizer(learning_rate)
    elif optimizer == 'sgd':
        return tf.train.GradientDescentOptimizer(learning_rate)
    elif soptimizer == 'rms':
        return tf.train.RMSPropOptimizer(learning_rate, decay=0.95, momentum=0.9)
    else:
        # assert is a good way to tell other how to use this function for bug happening.
        #-------
        # standard way to pring the error
        #-------
        assert False, 'optimizer must be adam, adagrad, sgd, or rms'

def update_parameters(loss, optimizer = 'adam'):
    '''
    It's for optimizing and logging training parameters
    
    1.using gradient clipping to avoid gradient explosion and vanishment.
    
    Gradient clipping is most common in recurrent neural networks. 
    When gradients are being propagated back in time, they can vanish 
    because they they are continuously multiplied by numbers less than one.
    This is called the vanishing gradient problem. 
    This is solved by LSTMs and GRUs, and if you’re using a deep feedforward network, 
    This is solved by residual connections. 
    On the other hand, you can have exploding gradients too. 
    This is when they get exponentially large from being multiplied by numbers larger 
    than 1. Gradient clipping will clip the gradients between two numbers to prevent them from getting too large.

    '''
    #---------------
    # setting
    #---------------
    grad_clip = 5 # Clip gradients elementwise to have norm at most equal to grad_clip.
    regularization_constant = 0.1 # Regularization constant applied to all trainable parameters.
    enable_parameter_averaging = False # If true, model saves exponential weighted averages of parameters to separate checkpoint file.
    global_step = tf.Variable(0, trainable = False) # Optional Variable to increment by one after the variables have been updated.
    learning_rate_var = tf.Variable(0.0, trainable = False)
    
    #----------------
    # for understanding regularization
    #----------------
    trainable_variables_1 = tf.trainable_variables()[0]
    square_1 = tf.square(trainable_variables_1)
    sum_1 = tf.reduce_sum(square_1)
    sqrt = tf.sqrt(sum_1)
    #-----------------
    # we can customized our regularization on the parameters we like
    #-----------------
    if regularization_constant != 0:
        # l2_norm: is a 0-D tensor. 
        # we do l2-norm on each trainable's parameters.
        l2_norm = tf.reduce_sum([tf.sqrt(tf.reduce_sum(tf.square(param))) for param in tf.trainable_variables()]) # Returns list including all variables created with trainable=True
        # the smaller the loss is, the better do finish overfitting 
        loss = loss + regularization_constant*l2_norm
    #-----------------
    # optimizing
    #-----------------
    # define the optimizer
    optimizer = get_optimizer(learning_rate_var, optimizer=optimizer)
    # compute grads: return A list of (gradient, variable) pairs. Variable is always present, but gradient can be None.
    grads = optimizer.compute_gradients(loss)
    # standard way to do gradient clipping
    clipped = [(tf.clip_by_value(g, -grad_clip, grad_clip), v_) for g, v_ in grads]
    step = optimizer.apply_gradients(clipped, global_step = global_step)
    print ('step - whtat optimizer.apply_gradients returns', step)
    #-----------------
    # if using moving average techniques
    #-----------------
    if enable_parameter_averaging:
        ema = tf.train.ExponentialMovingAverage(decay=0.995)
        maintain_averages_op = ema.apply(tf.trainable_variables())
        with tf.control_dependencies([step]):
            step = tf.group(maintain_averages_op)
    else:
        step = step
    return learning_rate_var


In [4]:
from tf_utils import temporal_convolution_layer
from tf_utils import time_distributed_dense_layer
from tf_utils import sequence_softmax_loss
from tf_utils import sequence_evaluation_metric
from data_utils import get_glove_vectors
from data_utils import load_vocab_and_return_word_to_id_dict
from tf_utils import shape_of_tensor


dim_word = 300 # lstm on word embeddings

trainable_embedding = False
USE_PRETRAINED = True
USE_CHARS = True
filename_words_vec = "../models/data/wordvec/word2vec.npz".format(dim_word)
filename_words_voc = "../models/data/wordvec/words_vocab.txt"
filename_chars_voc = "../models/data/wordvec/chars_vocab.txt"

nwords = len(load_vocab_and_return_word_to_id_dict(filename_words_voc))
nchars = len(load_vocab_and_return_word_to_id_dict(filename_chars_voc))
embeddings = (get_glove_vectors(filename_words_vec) if USE_PRETRAINED else None)
embeddings = embeddings.astype(np.float32)
enable_parameter_averaging = False

#----------
# for debugging
#-------------
max_seq_length = 36
max_word_length = 54
num_layers = 2
hidden_size_cnn = 300
dim_char = 100
k = 3
ntags = 3
# num residual block we used
hidden_size_char = 200
num_channels = [hidden_size_char,hidden_size_char] # 200 is hidden ouput unit of char
# causal
causal  = False
filter_widths = 5


In [ ]:
def sequence_crf_loss(y, y_hat, sequence_lengths, max_sequence_length):
    """
    Computes the log-likelihood of tag sequences in a CRF on variable length sequences.

    Args:
        y: Label tensor of shape [batch size, max_sequence_length], which should be indices of label.
        y_hat: Prediction tensor, [batch size, max_sequence_length, num_class], which should be unscaled score.
        sequence_lengths: Sequence lengths.  Tensor of shape [batch_size].
        max_sequence_length: maximum length of padded sequence tensor.

    Returns:
        batch_softmax_loss. 0-dimensional tensor.
        transition matrix.
    Reference of sparse_softmax_cross_entropy_with_logits:
        -Official: https://www.tensorflow.org/api_docs/python/tf/nn/sparse_softmax_cross_entropy_with_logits
        -Blog : https://blog.csdn.net/yc461515457/article/details/77861695
    """
    # compute log-likelihood between y(y_true) and y_hat(y_pred)
    log_likelihood, trans_params = tf.contrib.crf.crf_log_likelihood(y_hat, y, sequence_lengths) # log_likelihood with shape of [batch_size]
    batch_crf_loss = tf.reduce_mean(-log_likelihood)
    return batch_crf_loss, trans_params


In [16]:
from tf_utils import TemporalConvNet

with tf.Graph().as_default() as g:
    ####################################
    # Step1: get input_sequences 
    ####################################

    #------------
    # 1-D  
    #------------
    item_id = tf.placeholder(tf.int32, [None])
    history_length = tf.placeholder(tf.int32, [None]) # It's for arg of lstm model: sequence_length, == len(is_ordered_history)
    #------------   
    # 2-D  
    #------------
    word_id = tf.placeholder(tf.int32, [None, max_seq_length]) 
    label = tf.placeholder(tf.int32, [None, max_seq_length]) # [batch_size, num_class]
    if USE_CHARS:
        word_lengths = tf.placeholder(tf.int32, shape=[None, max_seq_length])
    #------------   
    # 3-D  
    #------------
    if USE_CHARS:
        char_ids = tf.placeholder(tf.int32, shape=[None, max_seq_length, max_word_length]) # [batch_size, max_seq_length, max_word_length]

    #------------
    # boolean parameter
    #------------
    keep_prob = tf.placeholder(tf.float32)
    is_training = tf.placeholder(tf.bool)
    
    #------------
    # word_embedding: get word embeddings matrix
    #------------
    if embeddings is None:
        logging.info('WARNING: randomly initializing word vectors')
        word_embeddings = tf.get_variable(
        shape = [nwords, dim_word],
        name = 'word_embeddings',
        dtype = tf.float32,
        )
    else:
        word_embeddings = tf.get_variable(
        initializer = embeddings, # it will hold the embedding
        trainable = trainable_embedding,
        name = 'word_embeddings',
        dtype = tf.float32
        )
    word_representation = tf.nn.embedding_lookup(params = word_embeddings, ids = word_id)
    #------------
    # char_embedding: get char embeddings matrix
    #------------
    if USE_CHARS:
        # get char embeddings matrix
        char_embeddings = tf.get_variable(
                shape=[nchars, dim_char],
                name="char_embeddings",
                trainable = True,
                dtype=tf.float32,
        )
        # get char_representation, 4-D, [batch_size, max_seq_length, max_word_length, dim_char]
        char_representation = tf.nn.embedding_lookup(params = char_embeddings, ids = char_ids)
        # convert 4-D into 3-D: put the timestep on axis=1 and should be charater-level axis
        s = tf.shape(char_representation) # 1-D tensor, (batch_size, max_seq_length, max_word_length, dim_char)
        char_representation = tf.reshape(char_representation, shape=[ s[0]*s[1], s[-2], dim_char]) # [batch_size * max_seq_length, max_word_length, dim_char]
        #---------------
        # CNN-based Word level representation from characters embeddings
        #---------------
        output = TemporalConvNet(inputs=char_representation, num_channels = num_channels, convolution_width= filter_widths, causal = causal)
        # max pooling on char-axis
        output = tf.reduce_max(output, axis=1)
        # reshape to word level representation
        word_representation_extracted_from_char = tf.reshape(output, shape=[s[0], s[1], hidden_size_char]) # [batch_size, max_seq_length, hidden_size_char]

    x = tf.concat([
    word_representation,
    word_representation_extracted_from_char
        ], axis = 2) # (?, 122, 300)

    ####################################
    # Step2: calculate_outputs 
    ####################################
    
    #-------------------------
    # NN architecuture-Simple CNN
    #-------------------------
    print ('Original features : {}'.format(x.shape))
    for i in range(num_layers):
        if i == 0:
            conv = temporal_convolution_layer(x , 
                                       output_units = hidden_size_cnn,
                                       convolution_width = k,
                                       dilated = False,
                                       causal = False,
                                       bias=True,
                                       activation=None, 
                                       dropout=None, 
                                       scope='cnn-{}'.format(i),
                                       reuse = False,
                                      )
        else:
            conv = temporal_convolution_layer(conv, 
                                       output_units = hidden_size_cnn,
                                       convolution_width = k,
                                       dilated = False,
                                       causal = False,
                                       bias=True,
                                       activation=None, 
                                       dropout=None,
                                       scope='cnn-{}'.format(i),
                                       reuse = False,
                                      )
            
        print ('CNN-{} layer : {}'.format(i, conv.shape))
    # output layer (linear)
    y_hat = time_distributed_dense_layer(conv, ntags, activation=None, scope='output-layer') # (?, 122, 3)
    print ('Output layer : {}'.format(y_hat.shape))
    print ('y_true : {}'.format(label.shape))
    #--------------
    # for second-level model
    #--------------
    prediction_tensors = {
        'item_id':item_id,
        'word_id':word_id,
        'final_states':conv, # 修改不要全部max_seq_lenghth都存, 只存到history_length的長度(save memory)
        'final_predictions':y_hat,
    }
    
    ####################################
    # Step3: calculate_loss +evaluation score+ optimizing
    ####################################
    loss = sequence_softmax_loss(y = label, y_hat = y_hat, sequence_lengths = history_length, max_sequence_length = max_seq_length)
    learning_rate_var  = update_parameters(loss)
    
    
    labels_pred = tf.cast(tf.argmax(y_hat, axis= 2),tf.int32) # (?, max_seq_length)
    log_likelihood, trans_params = tf.contrib.crf.crf_log_likelihood(y_hat,label, history_length)
    batch_crf_loss = tf.reduce_mean(-log_likelihood)
    score = sequence_evaluation_metric(y = label, y_hat = labels_pred, sequence_lengths = history_length, max_sequence_length = max_seq_length)['f1']

#     ####################################
#     # Step4: saving the model 
#     ####################################    
#     # create saver object
#     # max_to_keep: indicates the maximum number of recent checkpoint files to keep.
#     saver = tf.train.Saver(max_to_keep = 1)
#     if enable_parameter_averaging:
#         saver_averaged = tf.train.Saver(ema.variables_to_restore(), max_to_keep=1)    

    #-------------------------
    # standard
    #-------------------------
    init = tf.global_variables_initializer()

There is no need to use additional 1x1 convolution to solve discrepant input-output widths
Original features : (?, 36, 500)
CNN-0 layer : (?, 36, 300)
CNN-1 layer : (?, 36, 300)
Output layer : (?, 36, 3)
y_true : (?, 36)
step - whtat optimizer.apply_gradients returns name: "Adam"
op: "AssignAdd"
input: "Variable"
input: "Adam/value"
attr {
  key: "T"
  value {
    type: DT_INT32
  }
}
attr {
  key: "_class"
  value {
    list {
      s: "loc:@Variable"
    }
  }
}
attr {
  key: "use_locking"
  value {
    b: false
  }
}



In [17]:
import os
from collections import deque # for computing Train/validation losses are averaged over the last loss_averaging_window
import tensorflow as tf
warm_start_init_step = 0 # If nonzero, model will resume training a restored model beginning at warm_start_init_step.
batch_size = 128
loss_averaging_window = 10
num_validation_batches = 1
num_training_steps = 10
learning_rate=0.001
log_interval = 1
min_steps_to_checkpoint =1
early_stopping_steps = 10


base_dir = './'
checkpoint_dir = os.path.join(base_dir, 'checkpoints')

with tf.Session(graph=g, config = config) as sess:
    ####################################
    # 1. fit
    ####################################
    if warm_start_init_step:
        # continue the optimization at a recent checkpoint instead of having to restart the optimization from the beginning
        restore(warm_start_init_step)
        step = warm_start_init_step
    else:
        # start the optimization from the beginning
        sess.run(init) # Run the initializer
        step = 0
     
    log_likelihood_, batch_crf_loss_, trans_params_  = sess.run(fetches = [log_likelihood,batch_crf_loss, trans_params], feed_dict = {
                                                 word_id:sentence,
                                                 label: y_true,
                                                 history_length: length,
                                                 char_ids: char_id_____,
                                                 #word_lengths:word_lengths______
                
                                                    })


In [18]:
log_likelihood_.shape


(2,)

In [19]:
log_likelihood_

array([ -8.105577, -20.77645 ], dtype=float32)

In [20]:
trans_params_.shape

(3, 3)

In [21]:
trans_params_

array([[ 0.15558529, -0.9989271 ,  0.6235478 ],
       [-0.96864796, -0.9568076 ,  0.42102146],
       [ 0.7179775 ,  0.992568  ,  0.8633115 ]], dtype=float32)

In [23]:
batch_crf_loss_

14.441013

In [ ]:
# def conv1d(in_, filter_size, height, padding, is_train=True, drop_rate=0.0, scope=None):
#     with tf.variable_scope(scope or "conv1d"):
#         num_channels = in_.get_shape()[-1]
#         filter_ = tf.get_variable("filter", shape=[1, height, num_channels, filter_size], dtype=tf.float32)
#         bias = tf.get_variable("bias", shape=[filter_size], dtype=tf.float32)
#         strides = [1, 1, 1, 1]
#         in_ = tf.layers.dropout(in_, rate=drop_rate, training=is_train)
#         # [batch, max_len_sent, max_len_word / filter_stride, char output size]
#         xxc = tf.nn.conv2d(in_, filter_, strides, padding) + bias
#         out = tf.reduce_max(tf.nn.relu(xxc), axis=2)  # max-pooling, [-1, max_len_sent, char output size]
#         return out


# def multi_conv1d(in_, filter_sizes, heights, padding="VALID", is_train=True, drop_rate=0.0, scope=None):
#     with tf.variable_scope(scope or "multi_conv1d"):
#         assert len(filter_sizes) == len(heights)
#         outs = []
#         for i, (filter_size, height) in enumerate(zip(filter_sizes, heights)):
#             if filter_size == 0:
#                 continue
#             out = conv1d(in_, filter_size, height, padding, is_train=is_train, drop_rate=drop_rate,
#                          scope="conv1d_{}".format(i))
#             outs.append(out)
#         concat_out = tf.concat(axis=2, values=outs)
#         return concat_out


In [5]:
128*36*25

115200